# VARIABLES

In [ ]:
from brayns import Client, CircuitExplorer
from bluepy.v2 import Circuit, Simulation
from neurom.core.types import NeuriteType
import math
import time
import numpy

address = "127.0.0.1:5000"
FOCUS_FULL_MORPHOLOGY = 0
FOCUS_SOMA = 1
FOCUS_AXONS = 2
FOCUS_APICAL_DENDRITE = 3
FOCUS_BASAL_DENDRITE = 4
focus = FOCUS_FULL_MORPHOLOGY

#############################################################################
# MINIMUM REQUIRED PARAMS
# -----------------------
# We can create a txt file with a line per video and automate the generation
# Then call the script: $ python script.py <circuit path> <gid> <report> <result path>
#############################################################################
circuitPath = "/home/nadir/Data/CircuitTestData/share/BBPTestData/circuitBuilding_1000neurons/BlueConfig"
GID = 510
report = "somas"
# Will be used to store the video and the text file for the composser
resultPath = "/home/nadir/Desktop/sscxtest"

############################################################################
# Optional params, can be fixed
############################################################################
frameCachePath = "/home/nadir/Desktop/sscxtest"
generationResolution = [1920,1080]

# Temprary to make bluepy work on my personal computer
localCircuitPath = "/home/nadir/Data/CircuitTestData/share/BBPTestData/circuitBuilding_1000neurons/BlueConfig"

# =========================================================================================================

# INITIALIZE BLUEPY

In [ ]:
circuit = Circuit(localCircuitPath)
simulation = Simulation(localCircuitPath)

# ADJUST THE MOVIE FRAMES AROUND SPIKING TIME

In [ ]:
# Get cell spiking time
cellSpikeReport = simulation.spikes.get_gid(GID)

# Adjust the report recording time as much as 4 seconds (2 before the spike until 2 after)
reportObj = simulation.report(report)
reportStart = reportObj.meta['start_time']
reportEnd = reportObj.meta['end_time']
reportDt = reportObj.meta['time_step']
recordStart = max(reportStart, cellSpikeReport[0] - 2.0)
recordEnd = min(reportEnd, cellSpikeReport[0] + 2.0)

frameStart = int(math.floor(recordStart / reportDt))
frameEnd = int(math.floor(recordEnd / reportDt))

# INITIALIZE BRAYNS AND LOAD CIRCUIT

In [ ]:
#####################################################
# INITIALIZATION
#####################################################
brayns = Client(address)
ce = CircuitExplorer(brayns)

ap = brayns.get_application_parameters()
brayns.set_application_parameters(engine=ap["engine"],
                                  image_stream_fps=ap["image_stream_fps"],
                                  jpeg_compression=ap["jpeg_compression"],
                                  viewport=generationResolution)

#####################################################
# COMPUTE FOCUS POINTS
#####################################################
minBounds = [9999999.9] * 3
maxBounds = [-9999999.9] * 3

somaPoints = list()
apicalDendritePoints = list()
basalDendiritePoints = list()
axonsPoints = list()

def updateBounds(p, minBound, maxBound):
    x = p[0]
    y = p[1]
    z = p[2]

    if x < minBound[0]:
        minBound[0] = x
    elif x > maxBound[0]:
        maxBound[0] = x

    if y < minBound[1]:
        minBound[1] = y
    elif y > maxBound[1]:
        maxBound[1] = y

    if z < minBound[2]:
        minBound[2] = z
    elif z > maxBound[2]:
        maxBound[2] = z

for section in circuit.morph.get(GID, True).sections:
    points = section.points
    for p in points:
        x = p[0]
        y = p[1]
        z = p[2]
        
        updateBounds(p, minBounds, maxBounds)
            
        if section.type == NeuriteType.soma:
            somaPoints.append([x, y, z])
        elif section.type == NeuriteType.apical_dendrite:
            apicalDendritePoints.append([x, y, z])
        elif section.type == NeuriteType.axon:
            axonsPoints.append([x, y, z])
        elif section.type == NeuriteType.basal_dendrite:
            basalDendiritePoints.append([x, y, z])

minBoundSoma = [99999999.9] * 3
maxBoundSoma = [-99999999.9] * 3
if len(somaPoints) > 0:
    for p in somaPoints:
        updateBounds(p, minBoundSoma, maxBoundSoma)
else:
    print("No soma specific bounds")
    minBoundSoma = minBounds.copy()
    maxBoundSoma = maxBounds.copy()
    

minBoundADend = [99999999.9] * 3
maxBoundADend = [-99999999.9] * 3
if len(apicalDendritePoints) > 0:
    for p in apicalDendritePoints:
        updateBounds(p, minBoundADend, maxBoundADend)
else:
    print("No apical dendrite specific bounds")
    minBoundADend = minBounds.copy()
    maxBoundADend = maxBounds.copy()

minBoundAxon = [99999999.9] * 3
maxBoundAxon = [-99999999.9] * 3
if len(axonsPoints) > 0:
    for p in axonsPoints:
        updateBounds(p, minBoundAxon, maxBoundAxon)
else:
    print("No axon specific bounds")
    minBoundAxon = minBounds.copy()
    maxBoundAxon = maxBounds.copy()

minBoundBDend = [99999999.9] * 3
maxBoundBDend = [-99999999.9] * 3
if len(basalDendiritePoints):
    for p in basalDendiritePoints:
        updateBounds(p, minBoundBDend, maxBoundBDend)
else:
    print("No basal dendrite specific bounds")
    minBoundBDend = minBounds.copy()
    maxBoundBDend = maxBounds.copy()
    


#####################################################
# LOAD CIRCUIT
#####################################################

brayns.get_scene()
if len(brayns.scene.models) > 0:
    toRemove = list()
    for m in brayns.scene.models:
        toRemove.append(m["id"])
    brayns.remove_model(toRemove)

r = ce.load_circuit(path=circuitPath,
                circuit_color_scheme=CircuitExplorer.CIRCUIT_COLOR_SCHEME_NEURON_BY_ID,
                density=100.0,
                gids=[GID],
                targets=["mini50"],
                report="voltages",
                mesh_filename_pattern="mesh_{gid}.obj",
                load_soma=True,
                load_axon=True,
                load_dendrite=True,
                load_apical_dendrite=True,
                use_sdf=True,
                radius_multiplier=4.0)

modelLoaded = False
while not modelLoaded:
    # Forces model update
    brayns.get_model_properties(id=r["id"])

    # Get information about the model
    model = brayns.scene.models[0]
    aabb_min = model['bounds']['min']
    aabb_max = model['bounds']['max']

    aabb_diag = [0, 0, 0]
    for k in range(3):
        aabb_diag[k] = aabb_max[k] - aabb_min[k]

    # Compute camera height according to model size
    height = math.sqrt(aabb_diag[0]*aabb_diag[0] + aabb_diag[1]*aabb_diag[1] + aabb_diag[2]*aabb_diag[2])
    if math.isinf(height):
        # Model is not yet committed to the 3D scene (still loading...)
        time.sleep(1)
    else:
        modelLoaded = True


#####################################################
# FOCUS CAMERA
#####################################################

def boundsCenter(minB, maxB):   
    target = [0] * 3
    for i in range(3):
        target[i] = (minB[i] + maxB[i]) * 0.5
    return target

def focusCamera(minBound, maxBound):
    cam = brayns.get_camera()
    camParams = brayns.get_camera_params()

    target = boundsCenter(minBound, maxBound)

    modelYlen = maxBound[1] - minBound[1]
    fov = math.radians(camParams["fovy"] * 0.5)
    hipoLen = (modelYlen * 0.5) / math.sin(fov)
    dist = hipoLen * math.cos(fov)

    pos = target.copy()
    pos[2] = pos[2] + dist

    brayns.set_camera(current = cam["current"],
                      orientation = [0,0,0,1],
                      target=target,
                      position=pos,
                      types=cam["types"])

focus = FOCUS_FULL_MORPHOLOGY
if focus == FOCUS_FULL_MORPHOLOGY:
    focusCamera(minBounds, maxBounds)
elif focus == FOCUS_SOMA:
    focusCamera(minBoundSoma, maxBoundSoma)
elif focus == FOCUS_AXONS:
    focusCamera(minBoundAxon, maxBoundAxon)
elif focus == FOCUS_APICAL_DENDRITE:
    focusCamera(minBoundADend, maxBoundADend)
elif focus == FOCUS_BASAL_DENDRITE:
    focusCamera(minBoundBDend, maxBoundBDend)

#####################################################
# PROJECT POINTS OF INTEREST IN THE 2D PLANE
#####################################################

#def projectCenterPoint(point):
#    cam = brayns.get_camera()
#    camParams = brayns.get_camera_params()
#    
#    # Needed data for lookat camera matrix
#    eye = cam["position"].copy()
#    up = [0, 1, 0]
#    target = cam["target"].copy()
#    
#    # fov
#    fov = camParams["fovy"]
#    
#    # p is the position to project
#    
#    # Projection implementation here
#    
#
#
#projSoma = projectCenterPoint(boundsCenter(minBoundSoma, maxBoundSoma))
#projADend = projectCenterPoint(boundsCenter(minBoundADend, maxBoundADend))
#projBDend projectCenterPoint(boundsCenter(minBoundBDend, maxBoundBDend))
#projAxons = projectCenterPoint(boundsCenter(minBoundAxon, maxBoundAxon))
#
#f = open(resultPath+"/projections.txt", "a")
#f.write("soma="+str(projSoma)+"\n")
#f.write("adend="+str(projADend)+"\n")
#f.write("bdend="+str(projBDend)+"\n")
#f.write("axon="+str(projAxons)+"\n")
#f.flush()
#f.close()

#####################################################
# GENERATE THE FRAMES
#####################################################

animParams = brayns.get_animation_parameters()
animFrames = [i for i in range(frameStart, frameEnd, 1)]

cam = brayns.get_camera()
camParams = brayns.get_camera_params()

base = [cam["position"], [0,0,-1], [0,1,0], camParams["aperture_radius"], camParams["focus_distance"]]
camDefines = list()
for i in range(len(animFrames)):
        camDefines.append(base)

ce.export_frames_to_disk(path=frameCachePath, 
                         animation_frames=animFrames,
                         camera_definitions=camDefines)

progress = 0.0
while progress < 1.0:
    progress = ce.get_export_frames_progress()["progress"]
    time.sleep(1)

#####################################################
# GENERATE THE MOVIE
#####################################################

ce.make_movie(output_movie_path=resultPath + "/scripttest.mp4",
              fps_rate=8,
              frames_folder_path=frameCachePath)